In [3]:
from lxml import etree
import pandas as pd
from collections import Counter
import os
import glob
import re
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from numpy import array
import numpy as np


wdir = "/home/jose/Dropbox/biblia/tb/"
file = "TEIBible" # "*.xml"
outdir = "/home/jose/Dropbox/biblia/tb/resulting data/"

In [4]:
parser = etree.XMLParser(encoding='utf-8')

In [5]:
documento_xml = etree.parse(wdir+file+".xml", parser)

In [6]:
documento_root = documento_xml.getroot()

In [7]:
namespaces_concretos = {'tei':'http://www.tei-c.org/ns/1.0','xi':'http://www.w3.org/2001/XInclude'}

In [20]:
for book in documento_root.xpath('//tei:TEI', namespaces=namespaces_concretos, with_tail=True):
    title = book.xpath('.//tei:title[2]/tei:idno[@type="string"]/text()', namespaces=namespaces_concretos, with_tail=True)[0]
    if title == "ACT":
        print(title)
        rss = book.xpath(".//tei:rs/@key", namespaces=namespaces_concretos, with_tail=True)

        set_entities = list(set([entity for reference in rss for entity in reference.split(" ") if "pla" not in entity]))
        print(set_entities)


ACT
['#org115', '#per539', '#per488', '#per39', '#org6', '#per500', '#org102', '#per541', '#org124', '#per567', '#per494', '#org111', '#per101', '#per509', '#per50', '#per42', '#per480', '#org88', '#tim1', '#per545', '#per531', '#per542', '#per29', '#per528', '#per533', '#org3', '#per486', '#org65', '#per128', '#per517', '#org70', '#org5', '#org112', '#per4', '#org25', '#per557', '#per48', '#per495', '#per564', '#per555', '#per543', '#per21', '#per35', '#org78', '#per558', '#org15', '#per20', '#org100', '#org29', '#per104', '#per546', '#org8', '#per571', '#per14', '#per476', '#per496', '#per491', '#per149', '#per27', '#per492', '#per529', '#per534', '#per485', '#wor6', '#org27', '#per527', '#org22', '#per563', '#org12', '#per514', '#tim5', '#per17', '#per477', '#per504', '#per566', '#per519', '#per482', '#org11', '#tim17', '#per540', '#per556', '#per561', '#per11', '#org37', '#per570', '#per523', '#per552', '#org101', '#org119', '#per551', '#per568', '#per544', '#per522', '#org2', '#or

In [10]:
table_path ="/home/jose/Dropbox/biblia/tb/resulting data/TEIBible_ACT_q-rs@key-@toWhom-@who__matrix.csv"

In [16]:
matrix_units_entities_act = pd.read_csv(table_path, sep="\t", index_col=0)

In [17]:
matrix_units_entities_act.head()

,b.ACT.1.1,b.ACT.1.2,b.ACT.1.3,b.ACT.1.4,b.ACT.1.5,b.ACT.1.6,b.ACT.1.7,b.ACT.1.8,b.ACT.1.9,b.ACT.1.10,...,b.ACT.28.22,b.ACT.28.23,b.ACT.28.24,b.ACT.28.25,b.ACT.28.26,b.ACT.28.27,b.ACT.28.28,b.ACT.28.29,b.ACT.28.30,b.ACT.28.31
#per1359,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#per208,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#pla260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#pla364,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org80,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
places = sorted([entity for entity in matrix_units_entities_act.index.tolist() if "pla" in entity])

In [21]:
other_entities = sorted([entity for entity in matrix_units_entities_act.index.tolist() if "pla" not in entity])

In [22]:
matrix_entities_places = pd.DataFrame([], index=other_entities, columns=places).fillna(0)

In [23]:
matrix_entities_places.head()

,#pla0,#pla1,#pla100,#pla101,#pla102,#pla103,#pla104,#pla105,#pla106,#pla107,...,#pla90,#pla91,#pla92,#pla93,#pla94,#pla95,#pla96,#pla97,#pla98,#pla99
#ani0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
lista_vacia = []

In [41]:
listas = [['#per1', '#per476', '#wor5'],['#org3', '#per17', '#pla18', '#tim1']]

In [46]:
for lista in listas:
    if any("pla" in entity for entity in lista):
        print("yes")
    else:
        print("no")


no
yes


In [59]:
for textual_unit in matrix_units_entities_act.columns.tolist():
    entities_in_textual_unit_lt = sorted(matrix_units_entities_act[textual_unit][matrix_units_entities_act[textual_unit] > 0].index)
    
    if any("pla" in entity for entity in entities_in_textual_unit_lt):
        places_in_textual_unit_lt = [entity for entity in entities_in_textual_unit_lt if "pla" in entity]
        other_entities_in_textual_unit_lt = [entity for entity in entities_in_textual_unit_lt if "pla" not in entity]
        
        for entity in other_entities_in_textual_unit_lt:
            for place in places_in_textual_unit_lt:
                matrix_entities_places.loc[entity][place] += 1


In [60]:
matrix_entities_places.loc["#per1"].sort_values(ascending=False)

#pla3      20
#pla4      18
#pla8      12
#pla41     10
#pla80     10
#pla18      8
#pla2       6
#pla19      6
#pla51      6
#pla171     4
#pla30      4
#pla212     2
#pla70      2
#pla152     2
#pla164     2
#pla7       2
#pla178     2
#pla179     2
#pla180     2
#pla33      2
#pla63      2
#pla182     2
#pla15      2
#pla62      2
#pla214     2
#pla189     2
#pla199     2
#pla461     0
#pla362     0
#pla360     0
           ..
#pla592     0
#pla591     0
#pla590     0
#pla611     0
#pla613     0
#pla638     0
#pla627     0
#pla637     0
#pla636     0
#pla635     0
#pla634     0
#pla633     0
#pla632     0
#pla631     0
#pla630     0
#pla629     0
#pla628     0
#pla626     0
#pla614     0
#pla625     0
#pla624     0
#pla622     0
#pla621     0
#pla620     0
#pla619     0
#pla618     0
#pla617     0
#pla616     0
#pla615     0
#pla0       0
Name: #per1, Length: 850, dtype: int64

In [34]:
matrix_act.loc["#per1"].loc. > 0

b.ACT.1.1       True
b.ACT.1.2      False
b.ACT.1.3      False
b.ACT.1.4       True
b.ACT.1.5       True
b.ACT.1.6       True
b.ACT.1.7       True
b.ACT.1.8       True
b.ACT.1.9      False
b.ACT.1.10     False
b.ACT.1.11      True
b.ACT.1.12     False
b.ACT.1.13     False
b.ACT.1.14      True
b.ACT.1.15     False
b.ACT.1.16      True
b.ACT.1.17     False
b.ACT.1.18     False
b.ACT.1.19     False
b.ACT.1.20     False
b.ACT.1.21      True
b.ACT.1.22     False
b.ACT.1.23     False
b.ACT.1.24      True
b.ACT.1.25      True
b.ACT.1.26     False
b.ACT.2.1      False
b.ACT.2.2      False
b.ACT.2.3      False
b.ACT.2.4      False
               ...  
b.ACT.28.2     False
b.ACT.28.3     False
b.ACT.28.4     False
b.ACT.28.5     False
b.ACT.28.6     False
b.ACT.28.7     False
b.ACT.28.8     False
b.ACT.28.9     False
b.ACT.28.10    False
b.ACT.28.11    False
b.ACT.28.12    False
b.ACT.28.13    False
b.ACT.28.14    False
b.ACT.28.15    False
b.ACT.28.16    False
b.ACT.28.17    False
b.ACT.28.18  